In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q


In [2]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [3]:
# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

from pyspark.sql import SparkSession

#import functions/Classes for sparkml

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
spark = SparkSession.builder.appName("my-test-app").getOrCreate()

24/11/19 07:46:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark

In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv


--2024-11-19 07:46:37--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13891 (14K) [text/csv]
Saving to: ‘mpg.csv.2’

mpg.csv.2           100%[===================>]  13.57K  --.-KB/s    in 0s      

2024-11-19 07:46:37 (41.7 MB/s) - ‘mpg.csv.2’ saved [13891/13891]



In [7]:
df = spark.read.csv("mpg.csv", header=True, inferSchema=True)

In [8]:
df.show()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|15.0|        8|      390.0|       190|  3850|       8.5|  70|American|
|21.0|        6|      199.0|        90|  2648|      15.0|  70|American|
|18.0|        6|      199.0|        97|  2774|      15.5|  70|American|
|16.0|        8|      304.0|       150|  3433|      12.0|  70|American|
|14.0|        8|      455.0|       225|  3086|      10.0|  70|American|
|15.0|        8|      350.0|       165|  3693|      11.5|  70|American|
|18.0|        8|      307.0|       130|  3504|      12.0|  70|American|
|14.0|        8|      454.0|       220|  4354|       9.0|  70|American|
|15.0|        8|      400.0|       150|  3761|       9.5|  70|American|
|10.0|        8|      307.0|       200|  4376|      15.0|  70|American|
|15.0|        8|      383.0|       170|  3563|      10.0|  70|Am

In [9]:
df.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



In [10]:
assembler = VectorAssembler(inputCols=["Cylinders", "Engine Disp", "Horsepower", "Weight", "Accelerate", "Year"], outputCol="features")
transform_data = assembler.transform(df)

In [11]:
transform_data.select("Cylinders", "MPG").show()

+---------+----+
|Cylinders| MPG|
+---------+----+
|        8|15.0|
|        6|21.0|
|        6|18.0|
|        8|16.0|
|        8|14.0|
|        8|15.0|
|        8|18.0|
|        8|14.0|
|        8|15.0|
|        8|10.0|
|        8|15.0|
|        8|11.0|
|        8|10.0|
|        8|15.0|
|        6|21.0|
|        8|17.0|
|        8| 9.0|
|        8|14.0|
|        6|22.0|
|        8|14.0|
+---------+----+
only showing top 20 rows



In [12]:
# Split data into training and testing sets
(training_data, testing_data) = transform_data.randomSplit([0.7, 0.3], seed=42)

In [14]:
# Train linear regression model
# Ignore any warnings
lr = LinearRegression(featuresCol="features", labelCol="MPG")
model = lr.fit(training_data)

24/11/19 07:53:01 WARN util.Instrumentation: [21f56af7] regParam is zero, which might cause numerical instability and overfitting.


In [18]:
# Make predictions on testing data
predictions = model.transform(testing_data)

In [19]:
#R-squared (R2): R2 is a statistical measure that represents the proportion of variance
#in the dependent variable (target) that is explained by the independent variables (features).
#Higher values indicate better performance.

evaluator = RegressionEvaluator(labelCol="MPG", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R Squared =", r2)


R Squared = 0.7685170910359864


In [17]:
evaluator = RegressionEvaluator(labelCol="MPG", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("MAE =", mae)


MAE = 2.990444141294136


In [20]:
spark.stop()

In [24]:
spark = SparkSession.builder.appName("test-me-app").getOrCreate()
spark

In [25]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/diamonds.csv


--2024-11-19 08:12:08--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/diamonds.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192561 (3.0M) [text/csv]
Saving to: ‘diamonds.csv’

diamonds.csv        100%[===================>]   3.04M  --.-KB/s    in 0.05s   

2024-11-19 08:12:12 (56.0 MB/s) - ‘diamonds.csv’ saved [3192561/3192561]



In [27]:
mpg_data = spark.read.csv("mpg.csv", header=True, inferSchema=True)


In [30]:
mpg_data.show()

mpg_data.printSchema()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|15.0|        8|      390.0|       190|  3850|       8.5|  70|American|
|21.0|        6|      199.0|        90|  2648|      15.0|  70|American|
|18.0|        6|      199.0|        97|  2774|      15.5|  70|American|
|16.0|        8|      304.0|       150|  3433|      12.0|  70|American|
|14.0|        8|      455.0|       225|  3086|      10.0|  70|American|
|15.0|        8|      350.0|       165|  3693|      11.5|  70|American|
|18.0|        8|      307.0|       130|  3504|      12.0|  70|American|
|14.0|        8|      454.0|       220|  4354|       9.0|  70|American|
|15.0|        8|      400.0|       150|  3761|       9.5|  70|American|
|10.0|        8|      307.0|       200|  4376|      15.0|  70|American|
|15.0|        8|      383.0|       170|  3563|      10.0|  70|Am

In [ ]:
assembler = VectorAssembler(inputCols=["carat", "depth", "table"], outputCol="features")

transform_data =  assembler.transform(mpg_data)

In [ ]:
(training_data, testing_data) = diamond_transformed_data.randomSplit([0.7, 0.3])

In [ ]:
lr = LinearRegression(featuresCol="features", labelCol="price")
model = lr.fit(training_data)

In [ ]:
prediction = model.transform(testing_data)

In [ ]:
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R Squared =", r2)


evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("MAE =", mae)


evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE =", rmse)